# Candlestick and volume charts
Host an app on the local server that displays up-to-date historical charts the Poloniex cryptocurrency exchange.

In [ ]:
import datetime
import pandas as pd
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
from IPython import display

## Define functions for changing time scale for data. Define function for running dash inside Jupyter

In [2]:
def split_by_day(dataframe):
    df=dataframe.copy()
    df['day']=df.date.dt.date
    groups = df.groupby('day')
    
    day=[name for name,group in groups]
    open=[df.loc[group.date.min()].open for name,group in groups]
    high=[group.high.max() for name,group in groups]
    low=[group.low.min() for name,group in groups]
    close=[df.loc[group.date.max()].close for name,group in groups]
    volume=[group.volume.sum() for name,group in groups]
    weightedAverage=[group.weightedAverage.mean() for name,group in groups]

    return pd.DataFrame({'open':open, 'high':high, 'low':low, 'close':close, 'volume':volume},index=day)

def split_by_week(dataframe):

    df=dataframe.copy()
    df['week']=df.date.dt.week
    df['year']=df.date.dt.year
    groups = df.groupby(['year','week'])

    week=[datetime.datetime.strptime(str(name[0])+'-W'+str(name[1]) + '-0', "%Y-W%W-%w") for name,group in groups]
    
    open=[df.loc[group.date.min()].open for name,group in groups]
    high=[group.high.max() for name,group in groups]
    low=[group.low.min() for name,group in groups]
    close=[df.loc[group.date.max()].close for name,group in groups]
    volume=[group.volume.sum() for name,group in groups]
    weightedAverage=[group.weightedAverage.mean() for name,group in groups]

    return pd.DataFrame({'open':open, 'high':high, 'low':low, 'close':close, 'volume':volume},index=week)

def split_by_month(dataframe):
    
    df=dataframe.copy()
    df['month']=df.date.dt.month
    df['year']=df.date.dt.year
    groups = df.groupby(['year','month'])
    
    month=[str(name[0])+'-'+str(name[1]).zfill(2)+'-'+'01' for name,group in groups]
    open=[df.loc[group.date.min()].open for name,group in groups]
    high=[group.high.max() for name,group in groups]
    low=[group.low.min() for name,group in groups]
    close=[df.loc[group.date.max()].close for name,group in groups]
    volume=[group.volume.sum() for name,group in groups]
    weightedAverage=[group.weightedAverage.mean() for name,group in groups]

    return pd.DataFrame({'open':open, 'high':high, 'low':low, 'close':close, 'volume':volume},index=pd.to_datetime(month))

def show_app(app,  # type: dash.Dash
             port=9999,
             width=700,
             height=350,
             offline=True,
             style=True,
             **dash_flask_kwargs):

    url = 'http://localhost:%d' % port
    iframe = '<iframe src="{url}" width={width} height={height}></iframe>'.format(url=url, width=width, height=height)
    display.display_html(iframe, raw=True)
    if offline:
        app.css.config.serve_locally = True
        app.scripts.config.serve_locally = True
    if style:
        external_css = ["https://fonts.googleapis.com/css?family=Raleway:400,300,600",
                        "https://maxcdn.bootstrapcdn.com/font-awesome/4.7.0/css/font-awesome.min.css",
                        "http://getbootstrap.com/dist/css/bootstrap.min.css", ]

        for css in external_css:
            app.css.append_css({"external_url": css})

        external_js = ["https://code.jquery.com/jquery-3.2.1.min.js",
                       "https://cdn.rawgit.com/plotly/dash-app-stylesheets/a3401de132a6d0b652ba11548736b1d1e80aa10d/dash-goldman-sachs-report-js.js",
                       "http://getbootstrap.com/dist/js/bootstrap.min.js"]

        for js in external_js:
            app.scripts.append_script({"external_url": js})

    return app.run_server(debug=False,  # needs to be false in Jupyter
                          port=port,
                          **dash_flask_kwargs)

In [ ]:
app=dash.Dash()

app.layout=html.Div(children=[
        html.H1('Poloniex Chart History'),
    
        dcc.Dropdown(id='dropdown', placeholder='Please select a market...',
                     options=[
            {'label':'BTC-ETH', 'value':'BTC_ETH'},
            {'label':'BTC-LTC', 'value':'BTC_LTC'},
            {'label':'USDT-BTC', 'value':'USDT_BTC'},
            {'label':'BTC-XMR', 'value':'BTC_XMR'},            
            {'label':'BTC-XRP', 'value':'BTC_XRP'},
        ]
                    ),
        dcc.RadioItems(id='radio', options=[
            {'label':'4 hours', 'value':'4h'},
            {'label':'daily', 'value':'1d'},
            {'label':'weekly', 'value':'1w'},
            {'label':'monthly', 'value':'1m'}],
                       value='1w'
        ),

        html.Div(id='graph1'),
        html.Div(id='graph2')
])

@app.callback(
    Output(component_id='graph1', component_property='children'),
    [Input(component_id='dropdown', component_property='value'),
    Input(component_id='radio', component_property='value')]
)
def update_g1(input_data1,input_data2):

    
    try:
        df=pd.read_json('https://poloniex.com/public?command=returnChartData&currencyPair='+input_data1+'&start=1435699200&end=9999999999&period=14400')
    except:
        return dcc.Graph(id='failed')
    
    df=df.set_index(df['date'])
    df=df.drop(df.index[0:2])
    
    if input_data2 =='4h':
        df=df
    if input_data2 == '1d':
        df=split_by_day(df)
    elif input_data2 == '1w':
        df=split_by_week(df)
    elif input_data2 == '1m':
        df=split_by_month(df)

    
    candlestick = {
            'x': df.index,
            'open': df['open'],
            'high': df['high'],
            'low': df['low'],
            'close': df['close'],
            'type': 'candlestick',
            'increasing': {'line': {'color': 'green'}},
            'decreasing': {'line': {'color': 'red'}}
        }
    
    volume = {
            'x':df.index,
            'y':df.volume
    }

    return     dcc.Graph(id='out_graph1',
                        figure={'data':[candlestick],'layout':{'title':'Trading Data'}}
             ),dcc.Graph(id='out_graph2',
                        figure={'data':[volume],'layout':{'title':'Volume'}})

if __name__ == '__main__':
    app.run_server(debug=False, port = 8056)  # Run app on local server
    #show_app(app, port=8055, width = 1000, height =500) # Run app in Juypter
    

 * Running on http://127.0.0.1:8056/ (Press CTRL+C to quit)
127.0.0.1 - - [11/Jun/2018 22:04:23] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [11/Jun/2018 22:04:23] "GET /_dash-component-suites/dash_core_components/rc-slider@6.1.2.css?v=0.23.0 HTTP/1.1" 200 -
127.0.0.1 - - [11/Jun/2018 22:04:23] "GET /_dash-component-suites/dash_core_components/react-select@1.0.0-rc.3.min.css?v=0.23.0 HTTP/1.1" 200 -
127.0.0.1 - - [11/Jun/2018 22:04:23] "GET /_dash-component-suites/dash_core_components/react-virtualized@9.9.0.css?v=0.23.0 HTTP/1.1" 200 -
127.0.0.1 - - [11/Jun/2018 22:04:23] "GET /_dash-component-suites/dash_core_components/react-virtualized-select@3.1.0.css?v=0.23.0 HTTP/1.1" 200 -
127.0.0.1 - - [11/Jun/2018 22:04:23] "GET /_dash-component-suites/dash_core_components/react-dates@12.3.0.css?v=0.23.0 HTTP/1.1" 200 -
127.0.0.1 - - [11/Jun/2018 22:04:23] "GET /_dash-component-suites/dash_renderer/react@15.4.2.min.js?v=0.12.1 HTTP/1.1" 200 -
127.0.0.1 - - [11/Jun/2018 22:04:23] "GET /_dash-componen